### **Recuerden usar GPU en modelos open-source**

In [ ]:
!pip install transformers

In [3]:
from IPython.display import Markdown
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline

In [4]:
login(userdata.get('miHuggingFace'))

In [5]:
# Nombre del modelo:
# Usemos este de Llama de 7 mil millones de parámetros, entrenado para chateo:
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Cargar tokenizer y modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.float16,  #  -- puedes disminuirlo...
    device_map="auto"  # Esto distribuye automáticamente las capas según la GPU/CPU
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
def generate_response(prompt, max_new_tokens=1024):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [16]:
# Definimos los prompt de sistema:

system_prompt = """
Eres un consultor ambiental especializado en diseño de campañas sostenibles.
Debes responer de manera clara y simple. Si algo no sabes, no lo respondas, o
responde que no lo sabes.
Redacta el texto de salida en formato Markdown.
"""

In [9]:
# Prompt simple/directo (Zero-Shot):

user_prompt_zs = "Diseña una campaña para reducir el consumo de plástico."

In [10]:
# Construimos el prompt completo (system + user)
full_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt_zs} [/INST]"

# Generamos la respuesta del modelo
print("Iniciando proceso Zero-Shot...\n")

respuesta_completa = generate_response(full_prompt)

print("Resultado del Zero-Shot:")
print(respuesta_completa)
#Markdown(respuesta_completa)

Iniciando proceso Zero-Shot...

Resultado del Zero-Shot:
[INST] <<SYS>>

Eres un consultor ambiental especializado en diseño de campañas sostenibles.
Debes responer de manera clara y simple. Si algo no sabes, no lo respondas, o
comenta que no lo sabes.

<</SYS>>

Diseña una campaña para reducir el consumo de plástico. [/INST]  ¡Por supuesto! Para diseñar una campaña efectiva para reducir el consumo de plástico, aquí hay algunas estrategias que podrían ser útiles:

1. Educación y concienciación: Realizar una campaña de educación y concienciación para informar a la población sobre los riesgos del plástico y su impacto en el medio ambiente. Esto podría incluir la creación de materiales educativos, como folletos, carteles y videos, que expliquen cómo el plástico afecta al medio ambiente y cómo podemos reducir nuestro uso de plástico diario.
2. Alternativas sostenibles: Promover alternativas sostenibles al plástico, como bolsas reutilizables, cestas de supermercado, envases de madera o bici

In [11]:
Markdown(respuesta_completa)

[INST] <<SYS>>

Eres un consultor ambiental especializado en diseño de campañas sostenibles.
Debes responer de manera clara y simple. Si algo no sabes, no lo respondas, o
comenta que no lo sabes.

<</SYS>>

Diseña una campaña para reducir el consumo de plástico. [/INST]  ¡Por supuesto! Para diseñar una campaña efectiva para reducir el consumo de plástico, aquí hay algunas estrategias que podrían ser útiles:

1. Educación y concienciación: Realizar una campaña de educación y concienciación para informar a la población sobre los riesgos del plástico y su impacto en el medio ambiente. Esto podría incluir la creación de materiales educativos, como folletos, carteles y videos, que expliquen cómo el plástico afecta al medio ambiente y cómo podemos reducir nuestro uso de plástico diario.
2. Alternativas sostenibles: Promover alternativas sostenibles al plástico, como bolsas reutilizables, cestas de supermercado, envases de madera o bicicletas, y brindar incentivos económicos para sus usuarios.
3. Reducción de residuos: Implementar programas de recopilación y reciclaje de residuos de plástico en comunidades y empresas, y fomentar la separación de residuos para facilitar el reciclaje.
4. Políticas y leyes: Llevar a cabo campañas para influir en las políticas y leyes locales y nacionales para reducir el consumo de plástico. Esto podría incluir la creación de leyes que prohiban el uso de plástico en ciertos lugares o que impongan impuestos sobre el plástico no reutilizable.
5. Participación comunitaria: Organizar eventos y actividades en las comunidades para involucrar a la gente en la lucha contra el plástico. Esto podría incluir campañas de limpieza de playas, recolectas de plástico y creación de parques de plástico reciclado.
6. Campañas de marketing: Crear campañas de marketing innovadoras y efectivas para promover el uso de productos sostenibles y reducir el consumo de plástico.
7. Colaboración con empresas: Trabajar con empresas para desarrollar productos sostenibles y reducir el uso de plástico en sus procesos de producción y distribución.
8. Educación a través de la escuela: Educar a los niños y jóvenes sobre el impacto del plástico en el medio ambiente y cómo pueden contribuir a reducir su uso.
9. Incentivos económicos: Ofrecer incentivos económicos a las personas que utilizan productos sostenibles y reducen su uso de plástico.
10. Monitoreo y evaluación: Monitorear y evaluar el progreso de la campaña y ajustar estrategias según sea necesario.

Espero que estas estrategias te sean útiles para diseñar una campaña efectiva para reducir el consumo de plástico. Si tienes alguna pregunta adicional, no dudes en preguntar.

In [12]:
# Prompt usando Chain-of-Thought (CoT):

user_prompt_cot = """
Diseña una campaña para reducir el consumo de plástico.

Piensa paso a paso:
1. Define el problema.
2. Propón soluciones posibles.
3. Explica cómo implementarlas.
"""

In [13]:
# Construimos el prompt completo (system + user)
full_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt_cot} [/INST]"

# Generamos la respuesta del modelo
print("Iniciando proceso Chain-of-Thought...\n")

respuesta_completa = generate_response(full_prompt)

print("Resultado del CoT:")
Markdown(respuesta_completa)

Iniciando proceso Chain-of-Thought...

Resultado del CoT:


[INST] <<SYS>>

Eres un consultor ambiental especializado en diseño de campañas sostenibles.
Debes responer de manera clara y simple. Si algo no sabes, no lo respondas, o
comenta que no lo sabes.

<</SYS>>


Diseña una campaña para reducir el consumo de plástico.

Piensa paso a paso:
1. Define el problema.
2. Propón soluciones posibles.
3. Explica cómo implementarlas.
 [/INST]  ¡Por supuesto! Como consultor ambiental, estoy aquí para ayudarte a diseñar una campaña efectiva para reducir el consumo de plástico. Aquí te presento una posible estrategia paso a paso:

1. Define el problema:
El consumo excesivo de plástico es un problema ambiental grave que afecta a nuestro planeta y nuestros ecosistemas. El plástico no biodegradable puede durar cientos de años en la naturaleza, contaminando el suelo, el agua y el aire. Además, la producción de plástico requiere grandes cantidades de combustibles fósiles, lo que contribuye al cambio climático.
2. Propón soluciones posibles:
Para reducir el consumo de plástico, podemos implementar varias soluciones, como:
* Reducir el uso de plástico en la vida cotidiana: Evitar el uso de bolsas plásticas, botellas de plástico y otros productos que solo se utilizan una vez y luego se desechan. En su lugar, optar por productos sostenibles como bolsas de papel, botellas reutilizables de plástico, o incluso bolsas de madera o hule.
* Promover el reciclaje: Educar a la sociedad sobre la importancia del reciclaje y fomentar la separación de residuos. Esto puede incluir la creación de programas de reciclaje en las comunidades y en las empresas.
* Fomentar la reducción de residuos: Implementar programas de reducción de residuos en las empresas y en las comunidades, para reducir la cantidad de residuos generados y minimizar la cantidad de plástico que se desecha.
* Crear alternativas sostenibles: Desarrollar y promover productos y servicios alternativos que reemplazan a los productos de plástico, como productos de cuero, madera, hule, etc.
3. Explica cómo implementar las soluciones:
Para implementar estas soluciones, podemos seguir los siguientes pasos:
* Educación y sensibilización: Educar a la sociedad sobre la importancia de reducir el consumo de plástico y la necesidad de cambiar nuestras prácticas de consumo.
* Crear leyes y políticas: Implementar leyes y políticas que promuevan la reducción del consumo de plástico, como impuestos sobre el plástico, restricciones en la venta de productos de plástico, etc.
* Inversión en tecnología: invertir en tecnologías que permitan reducir el uso de plástico, como bolsas de comida reutilizables, botellas de plástico recicladas, etc.
* Participación comunitaria: involucrar a la comunidad en la implementación de estas soluciones, a través de programas de voluntariado, campañas de sensibilización, etc.

En resumen, para reducir el consumo de plástico debemos adoptar una abordación holística que incluya la educación, la creación de leyes y políticas, la inversión en tecnología y la participación comunitaria. Espero que esta información te haya sido útil para diseñar una campaña efectiva para reducir el consumo de plástico.

In [17]:
# Prompt usando Tree of Thoughts (ToT):

user_prompt_tot = """
Problema: Diseña una campaña para reducir el consumo de plástico.

Paso 1: Genera tres ideas distintas para una campaña:
1. [idea 1]
2. [idea 2]
3. [idea 3]

Paso 2: Evalúa cada idea según estos criterios:
- Impacto medioambiental
- Facilidad de implementación
- Capacidad de generar conciencia social

Paso 3: Selecciona la mejor idea o combina las mejores partes para proponer una solución final.
"""

In [18]:
# Construimos el prompt completo (system + user)
full_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt_tot} [/INST]"

# Generamos la respuesta del modelo
print("Iniciando proceso Tree-of-Thoughts...\n")

respuesta_completa = generate_response(full_prompt)

print("Resultado del ToT:")
Markdown(respuesta_completa)

Iniciando proceso Tree-of-Thoughts...

Resultado del ToT:


[INST] <<SYS>>

Eres un consultor ambiental especializado en diseño de campañas sostenibles.
Debes responer de manera clara y simple. Si algo no sabes, no lo respondas, o
responde que no lo sabes. 
Redacta el texto de salida en formato Markdown.

<</SYS>>


Problema: Diseña una campaña para reducir el consumo de plástico.

Paso 1: Genera tres ideas distintas para una campaña:
1. [idea 1]
2. [idea 2]
3. [idea 3]

Paso 2: Evalúa cada idea según estos criterios:
- Impacto medioambiental
- Facilidad de implementación
- Capacidad de generar conciencia social

Paso 3: Selecciona la mejor idea o combina las mejores partes para proponer una solución final.
 [/INST]  ¡Entendido! A continuación, te presento tres ideas distintas para una campaña para reducir el consumo de plástico, evaluadas según los criterios de impacto medioambiental, facilidad de implementación y capacidad de generar conciencia social:

Idea 1: "Deshazte de los plásticos"

* Impacto medioambiental: Alcanzar una reducción significativa del consumo de plástico y reducir la cantidad de residuos plásticos en el medio ambiente.
* Facilidad de implementación: Crear una campaña de recopilación de plásticos en comunidades y empresas, y organizar reciclajes en centros de reciclaje.
* Capacidad de generar conciencia social: Utilizar medios de comunicación social y publicidad para sensibilizar a la población sobre la importancia de reducir el consumo de plástico y la necesidad de reciclar.

Idea 2: "Reutiliza y reduce"

* Impacto medioambiental: Reducir el consumo de plástico en la producción y el transporte de productos, y promover la reutilización y el reciclaje de productos.
* Facilidad de implementación: Crear una campaña de educación y sensibilización en escuelas y empresas, y establecer programas de reutilización y reciclaje en comunidades y ciudades.
* Capacidad de generar conciencia social: Organizar eventos y actividades en línea y en persona para promover la reutilización y el reciclaje, y establecer partnerships con empresas y organizaciones para apoyar la campaña.

Idea 3: "Plástico no bueno"

* Impacto medioambiental: Reducir el consumo de plástico en la producción y el transporte de productos, y promover la sustitución de productos plásticos por productos sostenibles.
* Facilidad de implementación: Crear una campaña de educación y sensibilización en escuelas y empresas, y establecer programas de substitución de productos plásticos en comunidades y ciudades.
* Capacidad de generar conciencia social: Organizar eventos y actividades en línea y en persona para promover la sustitución de productos plásticos, y establecer partnerships con empresas y organizaciones para apoyar la campaña.

Como resultado de la evaluación, la mejor idea es combinar las mejores partes de las ideas 1 y 2 para proponer una solución final. La campaña podría titularse "Recicla y reduce: Un llamado a la acción para reducir el consumo de plástico" y incluir una combinación de estrategias de educación y sensibilización, reutilización y reciclaje, y sustitución de productos plásticos por productos sostenibles.

La campaña podría incluir acciones como:

* Crear una plataforma en línea para recopilar datos y medir el progreso de la campaña.
* Organizar eventos y actividades en línea y en persona para promover la reutilización y el reciclaje, y establecer programas de substitución de productos plásticos en comunidades y ciudades.
* Crear un programa de educación en escuelas y empresas para sensibilizar a la población sobre la importancia de reducir el consumo de plástico y la necesidad de reciclar.
* Establecer partnerships con empresas y organizaciones para apoyar la campaña y amplificar su impacto.

En resumen, la campaña "Recicla y reduce" es una solución efectiva para reducir el consumo de plástico y promover la sustentabilidad en la comunidad.